In [5]:
import pandas as pd
from tensorflow.keras.preprocessing import image

import os 

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.resnet_v2 import ResNet101V2

import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import models
from tqdm import tqdm
import numpy as np

In [2]:
df = pd.read_csv('./data/HAM10000_Metadata')

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

(Warning for Sunlight, increased risk of cancer) Actinic keratoses and intraepithelial carcinoma / Bowen's disease (__akiec__), 0

(Skin cancer) basal cell carcinoma (__bcc__), 1

(HARMLESS) benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, __bkl__), 2

(HARMLESS) dermatofibroma (__df__),  3

melanoma (__mel__),  4

(Not entirely harmless) melanocytic nevi (__nv__)   5

(HARMLESS) vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, __vasc__). 6 


More than 50% of lesions are confirmed through histopathology (histo), the ground truth for the rest of the cases is either follow-up examination (follow_up), expert consensus (consensus), or confirmation by in-vivo confocal microscopy (confocal). The dataset includes lesions with multiple images, which can be tracked by the lesion_id-column within the HAM10000_metadata file.

In [ ]:
# Number of cases where we can use LSTM

(df['lesion_id'].value_counts() > 1).sum()

In [ ]:
df.head()

In [ ]:
df = df.sample(frac = 1).reset_index()

df.head()

In [ ]:
df['localization'].unique()

In [ ]:
temp = []

for i in range(0, 10015):
    loc = df.iloc[i]['localization']
    if loc == 'abdomen':
        temp.append(tf.one_hot(0, 15))
    elif loc == 'scalp':
        temp.append(tf.one_hot(1, 15))
    elif loc == 'lower extremity':
        temp.append(tf.one_hot(2, 15))
    elif loc == 'trunk':
        temp.append(tf.one_hot(3, 15))
    elif loc == 'upper extremity':
        temp.append(tf.one_hot(4, 15))
    elif loc == 'back':
        temp.append(tf.one_hot(5, 15))
    elif loc == 'neck':
        temp.append(tf.one_hot(6, 15))
    elif loc == 'face':
        temp.append(tf.one_hot(7, 15))
    elif loc == 'chest':
        temp.append(tf.one_hot(8, 15))
    elif loc == 'foot':
        temp.append(tf.one_hot(9, 15))
    elif loc == 'ear':
        temp.append(tf.one_hot(10, 15))
    elif loc == 'unknown':
        temp.append(tf.one_hot(11, 15))
    elif loc == 'hand':
        temp.append(tf.one_hot(12, 15))
    elif loc == 'acral':
        temp.append(tf.one_hot(13, 15))
    elif loc == 'genital':
        temp.append(tf.one_hot(14, 15))

In [ ]:
one_hot_loc = np.array(temp)

one_hot_loc

In [3]:
# VGG-16
preprocessed = []

VGG_load = VGG16(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(VGG_load)
model.add(layers.Flatten())


for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    #ohl = one_hot_loc[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)

    preprocessed.append(preds) #np.concatenate((preds, ohl)))
    
df['VGG16'] = preprocessed

100%|████████████████████████████████████████████████████████████████████████████| 10015/10015 [05:51<00:00, 28.46it/s]


In [1]:
img.shape

NameError: name 'img' is not defined

In [ ]:
# Inception
preprocessed = []

IV3_load = InceptionV3(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(IV3_load)
model.add(layers.Flatten())

for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    ohl = one_hot_loc[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (299,299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    preds = model.predict(img).reshape(131072)

    preprocessed.append(np.concatenate((preds, ohl)))
    
df['IV3'] = preprocessed

In [ ]:
# ResNet
preprocessed = []

resnet_load = ResNet101V2(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(resnet_load)
model.add(layers.Flatten())

"""
for i in tqdm(range(0, 10015)):
    file = df.iloc[i]['image_id']
    ohl = one_hot_loc[i]    
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = model.predict(img).reshape(100352)

    preprocessed.append(np.concatenate((preds, ohl)))
    
df['resnet'] = preprocessed
"""

In [ ]:
labels = []
for i in range(0, 10015):
    dx = df.iloc[i]['dx']
    if dx == 'akiec':
        labels.append(tf.one_hot(0, 7))
    elif dx == 'bcc':
        labels.append(tf.one_hot(1, 7))
    elif dx == 'bkl':
        labels.append(tf.one_hot(2, 7))
    elif dx == 'df':
        labels.append(tf.one_hot(3, 7))
    elif dx == 'mel':
        labels.append(tf.one_hot(4, 7))
    elif dx == 'nv':
        labels.append(tf.one_hot(5, 7))
    elif dx == 'vasc':
        labels.append(tf.one_hot(6, 7))
        
df['one_hot'] = labels

In [ ]:
df.head()

(Warning for Sunlight, increased risk of cancer) Actinic keratoses and intraepithelial carcinoma / Bowen's disease (__akiec__), 0

(Skin cancer) basal cell carcinoma (__bcc__), 1

(HARMLESS) benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, __bkl__), 2

(HARMLESS) dermatofibroma (__df__),  3

melanoma (__mel__),  4

(Not entirely harmless) melanocytic nevi (__nv__)   5

(HARMLESS) vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, __vasc__). 6 


More than 50% of lesions are confirmed through histopathology (histo), the ground truth for the rest of the cases is either follow-up examination (follow_up), expert consensus (consensus), or confirmation by in-vivo confocal microscopy (confocal). The dataset includes lesions with multiple images, which can be tracked by the lesion_id-column within the HAM10000_metadata file.

In [61]:
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append(0.0)
    elif df.iloc[i]['dx'] == 'nv':
        risk.append(1.0)
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append(2.0)
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append(3.0)
        
df['risk'] = risk

df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,VGG16,risk
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"[0.0, 0.0, 11.025155, 0.0, 0.08701746, 0.0, 0....",0.0
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"[0.0, 0.0, 1.2555947, 0.72406036, 0.0, 0.0, 0....",0.0
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"[0.0, 0.0, 6.3895617, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0


In [ ]:
df[df['risk'] == 3.0]['dx'].value_counts()

In [ ]:
df["dx"].value_counts()

In [ ]:
df["risk"].value_counts()

In [63]:
import pickle

In [ ]:
#Save VGG without body parts
pickle_out = open("VGG16_no_body.pickle", "wb")
pickle.dump(df["VGG16"], pickle_out)
pickle_out.close()

In [ ]:
#Load VGG without body parts
pickle_in = open("VGG16_no_body.pickle", "rb")
z_vgg_no_body = pickle.load(pickle_in)

In [ ]:
pickle_out = open("VGG16.pickle", "wb")
pickle.dump(df["VGG16"], pickle_out)
pickle_out.close()

pickle_out = open("IV3.pickle", "wb")
pickle.dump(df["IV3"], pickle_out)
pickle_out.close()

pickle_out = open("resnet.pickle", "wb")
pickle.dump(df["resnet"], pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("one_hot.pickle", "wb")
pickle.dump(df["one_hot"], pickle_out)
pickle_out.close()

pickle_out = open("risk.pickle", "wb")
pickle.dump(df["risk"], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("VGG16.pickle", "rb")
z_vgg = pickle.load(pickle_in)

pickle_in = open("IV3.pickle", "rb")
z_iv3 = pickle.load(pickle_in)

pickle_in = open("resnet.pickle", "rb")
z_res = pickle.load(pickle_in)

In [64]:
pickle_in = open("one_hot.pickle", "rb")
z_one_hot = pickle.load(pickle_in)

pickle_in = open("risk.pickle", "rb")
z_risk = pickle.load(pickle_in)

In [ ]:
vg_z = np.array(z_vgg)
iv_z = np.array(z_iv3)
res_z = np.array(z_res)
one_hot_z = np.array(z_one_hot)
risk_z = np.array(z_risk)

In [ ]:
df["one_hot"]

In [ ]:
VGG_X.shape

In [62]:
VGG_X = np.array(df["VGG16"])
#IV3_X = np.array(z_iv3)
#RES_X = np.array(z_res)

In [65]:
#y = np.array(df['one_hot'])

y = np.array(z_risk)

In [66]:
new_y = []
for ele in y:
    new_y.append(tf.one_hot(int(ele), 4))
    
new_y = np.array(new_y)

new_y.shape

(10015, 4)

In [ ]:
new_y

In [67]:
vgg_model = models.Sequential()
#vgg_model.add(layers.Dense(8192, activation='relu'))
#vgg_model.add(layers.Dense(4096, activation='relu'))
#vgg_model.add(layers.Dense(2048, activation='relu'))
vgg_model.add(layers.Dense(1024, activation='relu',input_shape = [25088]))
vgg_model.add(layers.Dense(512, activation='relu'))
vgg_model.add(layers.Dense(128, activation='relu'))
vgg_model.add(layers.Dense(4))

In [ ]:
#new_VGG = np.asarray(new_VGG).astype('float32')

In [ ]:
new_y

In [68]:
VGG_X.shape

new_VGG = []
for i in range(0, 10015):
    new_VGG.append(VGG_X[i])
    
new_VGG = np.array(new_VGG)
new_VGG.shape

(10015, 25088)

In [ ]:
new_RES = []
for i in range(0, 10015):
    new_RES.append(RES_X[i])
    
new_RES = np.array(new_RES)
new_RES.shape

In [ ]:
new_IV3 = []
for i in range(0, 10015):
    new_IV3.append(IV3_X[i])
    
new_IV3 = np.array(new_IV3)
new_IV3.shape

In [69]:
vgg_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])

vgg_model.fit(new_VGG, new_y, epochs=10, validation_split=0.1)

Train on 9013 samples, validate on 1002 samples
Epoch 1/10
9013/9013 [==============================] - 3s 280us/sample - loss: 2.3665 - accuracy: 0.5930 - val_loss: 1.0121 - val_accuracy: 0.6527
Epoch 2/10
9013/9013 [==============================] - 2s 240us/sample - loss: 0.9012 - accuracy: 0.6756 - val_loss: 1.0545 - val_accuracy: 0.6477
Epoch 3/10
9013/9013 [==============================] - 2s 239us/sample - loss: 0.7666 - accuracy: 0.7157 - val_loss: 1.1179 - val_accuracy: 0.5858
Epoch 4/10
9013/9013 [==============================] - 2s 243us/sample - loss: 0.5896 - accuracy: 0.7807 - val_loss: 1.4790 - val_accuracy: 0.5818
Epoch 5/10
9013/9013 [==============================] - 2s 251us/sample - loss: 0.4364 - accuracy: 0.8388 - val_loss: 1.6604 - val_accuracy: 0.6267
Epoch 6/10
9013/9013 [==============================] - 2s 247us/sample - loss: 0.3247 - accuracy: 0.8804 - val_loss: 1.8702 - val_accuracy: 0.5150
Epoch 7/10
9013/9013 [==============================] - 2s 244us

In [75]:
vgg_model.save("VGG_Model_NoBody.h5")

In [ ]:
loss = vgg_model.history.history['loss']
val_loss = vgg_model.history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('VGG_Loss.png', dpi = 300)
plt.show()

In [ ]:
plt.clf()
acc = vgg_model.history.history['accuracy']
val_acc = vgg_model.history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.savefig('VGG_Acc.png', dpi = 300)
plt.show()

In [ ]:
res_model = models.Sequential()
#res_model.add(layers.Dense(8192, activation='relu'))
#res_model.add(layers.Dense(4096, activation='relu'))
#res_model.add(layers.Dense(2048, activation='relu'))
res_model.add(layers.Dense(1024, activation='relu', input_shape = [100367]))
res_model.add(layers.Dense(512, activation='relu'))
res_model.add(layers.Dense(128, activation='relu'))
res_model.add(layers.Dense(4))

In [ ]:
res_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
res_model.fit(new_RES, new_y, epochs=20, validation_split=0.1)

In [ ]:
loss = res_model.history.history['loss']
val_loss = res_model.history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('RES_Loss.png', dpi = 300)
plt.show()

In [ ]:
plt.clf()
acc = res_model.history.history['accuracy']
val_acc = res_model.history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.savefig('RES_Acc.png', dpi = 300)
plt.show()

In [ ]:
df.groupby("risk")["localization"].value_counts()

In [ ]:
iv3_model = models.Sequential()
#iv3_model.add(layers.Dense(8192, activation='relu'))
#iv3_model.add(layers.Dense(4096, activation='relu'))
#iv3_model.add(layers.Dense(2048, activation='relu'))
iv3_model.add(layers.Dense(1024, activation='relu', input_shape = [131087]))
iv3_model.add(layers.Dense(512, activation='relu'))
iv3_model.add(layers.Dense(128, activation='relu'))
iv3_model.add(layers.Dense(4))

In [ ]:
iv3_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
iv3_model.fit(new_IV3, new_y, epochs=20, validation_split=0.1)

In [ ]:
loss = iv3_model.history.history['loss']
val_loss = iv3_model.history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('IV3_Loss.png', dpi = 300)
plt.show()

In [ ]:
plt.clf()
acc = iv3_model.history.history['accuracy']
val_acc = iv3_model.history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.savefig('IV3_Acc.png', dpi = 300)
plt.show()

In [ ]:
pickle_in = open("VGG_weight.pkl", "rb")
VGG_weight = pickle.load(pickle_in)

In [ ]:
iv3_model

In [ ]:
vgg_test = models.Sequential()
#vgg_model.add(layers.Dense(8192, activation='relu'))
#vgg_model.add(layers.Dense(4096, activation='relu'))
#vgg_model.add(layers.Dense(2048, activation='relu'))
vgg_test.add(layers.Dense(1024, activation='relu',input_shape = [25088]))
vgg_test.add(layers.Dense(512, activation='relu'))
vgg_test.add(layers.Dense(128, activation='relu'))
vgg_test.add(layers.Dense(4))

In [ ]:
vgg_test.set_weights(VGG_weight)

In [133]:
img = image.load_img(file_path, target_size= (224,224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = tf.keras.applications.vgg16.preprocess_input(img)
preds = model.predict(img).reshape(25088)

In [134]:
preds.shape

(25088,)

In [97]:
file_path = "ISIC_0024306.JPG"

In [98]:
Model = tf.keras.models.load_model("VGG_Model_NoBody.h5", compile = False)

In [143]:
def model_predict(img_path, Model):
    #load preprocessing model
    VGG_load = VGG16(weights='imagenet', include_top=False)
    model = models.Sequential()
    model.add(VGG_load)
    model.add(layers.Flatten())
    
    #image preprocessing 
    img = image.load_img(img_path, target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)
    
    #predict cancer type
    preds = Model.predict(np.array([preds]))
    return preds[0]

In [144]:
lesion_classes_dict = {
    0 : 'no risk',
    1 : 'low risk',
    2 : 'medium risk',
    3 : 'high risk'
    }

In [145]:
preds = model_predict(file_path , Model)

In [146]:
preds

array([-5.2549977, -1.1524458,  7.3475866, -3.7016683], dtype=float32)

In [148]:
pred_class = preds.argmax(axis=-1)
pr = lesion_classes_dict[pred_class]
result =str(pr)

In [149]:
result

'medium risk'